In [ ]:
%pip install tiktoken sentence-transformers

In [ ]:
database ="PPW"

In [ ]:
%reload_ext dotenv
%dotenv /notebooks/chainlang/my.env

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:

llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [ ]:
db  = Chroma(persist_directory=database, embedding_function=embedding )

In [ ]:
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

AS QA

In [ ]:
from langchain.prompts import PromptTemplate

template = """Conversation so far:
{chat_history}


Act as: Python Developer

Technology stack: LangChain

HUMAN:{question}
AI:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory,   condense_question_prompt=QA_CHAIN_PROMPT,)

In [ ]:
qa_chain({ "question" : "how can a create character-based memory for Langchain conversation chain" }) 

In [ ]:
import os
os.environ["DISABLE_NEST_ASYNCIO"]="True"

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response =  qa_chain({ "new_message" : message }) 
    return gpt_response["answer"]

gr.ChatInterface(predict).launch(share=True)

AS CONVERSATION

In [ ]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)
import math
import faiss
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.memory import ConversationSummaryMemory, VectorStoreRetrieverMemory,CombinedMemory

def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return score


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Initialize the vectorstore as empty
    embedding =  HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large"
    )
    embedding_size = 1024
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embedding, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
    return VectorStoreRetrieverMemory(retriever=retriever,input_key="text",memory_key="chat_history")

In [ ]:
from langchain.memory import ConversationBufferMemory


from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.chains import ConversationChain  
from langchain.memory import ConversationSummaryMemory, CombinedMemory,VectorStoreRetrieverMemory

template=f'''
You are an helpful developing assitant
'''
memory_template = """
Context:
{context}
Summary of the chat:
{chat_summary}

"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    AIMessagePromptTemplate.from_template("Alright, let's get started..."),
    SystemMessagePromptTemplate.from_template(memory_template),
    human_message_prompt])


In [ ]:
context_memory = VectorStoreRetrieverMemory(retriever=retriever,input_key="text",memory_key="context")

In [ ]:
summary_memory = ConversationSummaryMemory(llm=llm,memory_key="chat_summary",return_messages=False,input_key="text")

memory = CombinedMemory(memories=[context_memory, summary_memory])
conversation = LLMChain(
    llm=llm,
    prompt=chat_prompt,
    verbose=True,
    memory=memory
)


In [ ]:
conversation({ "text" : "how can a create character-based memory for Langchain conversation chain" }) 

In [ ]:
import os
os.environ["DISABLE_NEST_ASYNCIO"]="True"

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response =  conversation({ "text" : message }) 
    return gpt_response["text"]

gr.ChatInterface(predict).launch(share=True)

AS TOOL

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever, 
    "documentation_langchain",
    "Searches and returns update to date documentation about langchain",
)
tools = [tool]

In [ ]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)



In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    result =agent_executor({"input": message})
    return result['output']

gr.ChatInterface(predict).launch(share=True)